# Lab | Advanced PostgreSQL 

## Introduction

In this lab you will practice postgreSQL subqueries, temporary tables, and action queries.  
- Use the same database from the lab `postgreSQL_select`. If you are starting now, use the code from `publications_database.sql` to create your databese, tables and insert data. 

In [1]:
import pandas as pd

In [2]:
import sqlalchemy as db

In [3]:
# for python-dotenv method

from dotenv import load_dotenv
load_dotenv()
import os

In [4]:
# PostgreSQL

db_server = "postgresql"
db_user = "postgres"
db_password = os.environ.get('PASSWORD')
db_host = "localhost"
db_database = "w09_06_lab"
db_port = 5432

# create the engine
engine = db.create_engine(
    f"{db_server}://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"
)

# open the connection
conn = engine.connect()

# Close the connection
# conn.close()

In [5]:
conn

In [6]:
print('authors')
print(pd.read_sql_table("authors", conn).head(2))
print('')
print('titles')
print(pd.read_sql_table("titles", conn).head(2))
print('')
print('publishers')
print(pd.read_sql_table("publishers", conn).head(2))
print('')
print('sales')
print(pd.read_sql_table("sales", conn).head(2))
print('')
print('titleauthor')
print(pd.read_sql_table("titleauthor", conn).head(2))
print('')

authors
         au_id au_lname  au_fname         phone            address  \
0  172-32-1176    White   Johnson  408 496-7223    10932 Bigge Rd.   
1  213-46-8915    Green  Marjorie  415 986-7020  309 63rd St. #411   

         city state    zip  contract  
0  Menlo Park    CA  94025         1  
1     Oakland    CA  94618         1  

titles
  title_id                                              title      type  \
0   BU1032                The Busy Executive's Database Guide  business   
1   BU1111  Cooking with Computers: Surreptitious Balance ...  business   

   pub_id    price    advance royalty ytd_sales  \
0    1389  19.9900  5000.0000      10      4095   
1    1389  11.9500  5000.0000      10      3876   

                                               notes              pubdate  
0  An overview of available database systems with...  1991-06-12 00:00:00  
1  Helpful hints on how to use your electronic re...  1991-06-09 00:00:00  

publishers
   pub_id          pub_name        c

**1. Most profiting authors**

In this challenge you'll find out *who are the top 3 most profiting authors* in the `publications_database`.   

*It is possible to solve this exercise using one query, to do that you will need to use subqueries. We strongly recommend follow the steps below.*  

We will split this challenge in 3 steps:


# **Step 1: Calculate the `royalty of each sale for each author` and the `advance for each author and publication`.**

Write a `SELECT` query to obtain the following columns:

- `titleauthor.au_id `. Rename this field as AuthorID
- `sales.title_id`. Rename this field as TitleID
- Royalty of each sale. Rename this field as `royalty_per_sale` Use the formula:
    ```
    titles.price * sales.qty * (titles.royalty / 100) * (titleauthor.royaltyper / 100)
    ```
- Advance of each title and author. Rename this field as `advance_per_title_and_author`. Use the formula: 
    ```
    titles.advance * (titleauthor.royaltyper / 100)
    ```
- Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

*Note: In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sale.*

Your output should look something like below:

![Step 1 output](./images/step-1.png)

After your solution use the query to create a temporary table called `step1_royalties`.

In [7]:
query = '''
DROP TABLE IF EXISTS step1_royalties;
CREATE TEMP TABLE step1_royalties AS
SELECT 
    titleauthor.au_id AS author_ID, 
    sales.title_id AS title_ID,
    (CAST(titles.price AS float) * sales.qty) * (CAST(titles.royalty AS float) / 100) * (titleauthor.royaltyper / 100) AS royalty_per_sale,
    CAST(titles.advance AS float) * (titleauthor.royaltyper / 100) AS advance_per_title_and_author
FROM 
    sales
JOIN 
    titles
    ON sales.title_id = titles.title_id
INNER JOIN
    titleauthor 
    ON titles.title_id = titleauthor.title_id;
SELECT * FROM step1_royalties;
'''
pd.read_sql(query, conn)

,author_id,title_id,royalty_per_sale,advance_per_title_and_author
0,409-56-7008,BU1032,0.000,0.0
1,213-46-8915,BU1032,0.000,0.0
2,998-72-3567,PS2091,0.000,0.0
3,899-46-2035,PS2091,0.000,0.0
4,846-92-7186,PC8888,0.000,0.0
5,427-17-2319,PC8888,0.000,0.0
6,998-72-3567,PS2091,0.000,0.0
7,899-46-2035,PS2091,0.000,0.0
8,998-72-3567,PS2091,0.000,0.0
9,899-46-2035,PS2091,0.000,0.0


# **Step 2: Aggregate the total royalties for each title and author.**

Using the temporary table from Step 1, write a query to obtain the following output:

- AuthorID
- TitleID
- SUM of royalty_per_sale
- advance_per_title_and_author
  
 Hint: To use the *SUM* you should *GROUP BY* `AuthorID` and `TitleID`.

*Note: In the output of this step, each title should appear only once for each author.*  


Your output should look something like below:

![Step 2 output](./images/step-2.png)

After your solution use the query to create a temporary table called `step2_royalties`.

In [8]:
query = '''
DROP TABLE IF EXISTS step2_royalties;
CREATE TEMP TABLE step2_royalties AS
SELECT
    author_id,
    title_id,
    SUM(royalty_per_sale) as royalty_per_sale_and_author,
    advance_per_title_and_author
FROM
    step1_royalties
GROUP BY
    author_id,
    title_id,
    advance_per_title_and_author;
SELECT * FROM step2_royalties;
'''
pd.read_sql(query, conn)

,author_id,title_id,royalty_per_sale_and_author,advance_per_title_and_author
0,672-71-3249,TC7777,0.000,0.0
1,213-46-8915,BU2075,25.116,10125.0
2,846-92-7186,PC8888,0.000,0.0
3,172-32-1176,PS3333,29.985,2000.0
4,472-27-2349,TC7777,0.000,0.0
5,998-72-3567,PS2091,0.000,0.0
6,998-72-3567,PS2106,17.500,6000.0
7,899-46-2035,PS2091,0.000,0.0
8,712-45-1867,MC2222,23.988,0.0
9,274-80-9391,BU7832,29.985,5000.0


# **Step 3: Calculate the total profits of each author.**

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the temporary table from Step 2, write a query that contains the following output:

- Author ID
- SUM of profits of each author aggregating by `royalty_per_sale_and_author` and `advance_per_title_and_author`.

Sort the output based on a total profits from high to low, and limit the number of rows to 3.


Your output should look something like below:

![Step 3 output](./images/step-3.png)

In [9]:
query = '''
DROP TABLE IF EXISTS step3_royalties;
CREATE TEMP TABLE step3_royalties AS

SELECT
    author_id,
    ROUND(SUM(royalty_per_sale_and_author + advance_per_title_and_author)) AS profits
FROM
    step2_royalties
GROUP BY
    author_id;
    
SELECT
    *
FROM
    step3_royalties
ORDER BY 
    profits DESC
LIMIT 3;
'''
pd.read_sql(query, conn)

,author_id,profits
0,213-46-8915,10150.0
1,238-95-7766,7110.0
2,807-91-6654,7084.0


**2. Alternative solution**

In the previous challenge, you have developed your solution creating temporary tables and query the temporary tables in the subsequent steps.
We'd like you to try using subqueries. you can check a documentation [here](https://dev.mysql.com/doc/refman/8.0/en/derived-tables.html)

In [10]:
query = '''
SELECT
    author_id,
    ROUND(SUM(royalty_per_sale_and_author + advance_per_title_and_author)) AS profits
FROM
    (SELECT
        author_id,
        title_id,
        SUM(royalty_per_sale) as royalty_per_sale_and_author,
        advance_per_title_and_author
    FROM
        (SELECT 
            titleauthor.au_id AS author_ID, 
            sales.title_id AS title_ID,
            (CAST(titles.price AS float) * sales.qty) * (CAST(titles.royalty AS float) / 100) * (titleauthor.royaltyper / 100) AS royalty_per_sale,
            CAST(titles.advance AS float) * (titleauthor.royaltyper / 100) AS advance_per_title_and_author
        FROM 
            sales
        INNER JOIN 
            titles
            ON sales.title_id = titles.title_id
        INNER JOIN
            titleauthor 
            ON titles.title_id = titleauthor.title_id) AS step_one
    GROUP BY
        author_id,
        title_id,
        advance_per_title_and_author) AS step_two
GROUP BY
    author_id
ORDER BY 
    profits DESC
LIMIT 3;
'''
pd.read_sql(query, conn)

,author_id,profits
0,213-46-8915,10150.0
1,238-95-7766,7110.0
2,807-91-6654,7084.0
